# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Applications/Data Analysis Bootcamp/UofM-VIRT-DATA-PT-06-2024-U-LOLC/02_Challenges/06_APIs/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sola,-13.8833,167.5500,25.37,69,75,7.72,VU,1725517046
1,1,aral,40.5156,81.2636,26.59,20,4,3.57,CN,1725517047
2,2,sadovoye,47.7764,44.5089,23.31,38,25,7.66,RU,1725517048
3,3,adamstown,-25.0660,-130.1015,18.44,59,74,6.68,PN,1725517049
4,4,nova sintra,14.8667,-24.7167,26.32,85,100,3.05,CV,1725517050


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=False,  # Set to False if you're not using geographic projection
    size='Humidity', 
    color='Humidity', 
    title='City Humidity',
    xlabel='Longitude',
    ylabel='Latitude',
    frame_width=800,
    frame_height=600,
    cmap='Blues'
)

# Display the map
map_plot

:Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] >= 20) & 
    (city_data_df["Max Temp"] <= 25) & 
    (city_data_df["Wind Speed"] <= 10) & 
    (city_data_df["Cloudiness"] <= 20)
]
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,priozersk,61.0393,30.1291,22.75,78,1,0.79,RU,1725517061
31,31,tongliao,43.6125,122.2653,23.12,34,9,6.47,CN,1725517083
36,36,kone,-21.0595,164.8658,23.05,44,1,3.76,NC,1725517088
38,38,sao vicente,-23.9631,-46.3919,22.59,56,0,1.59,BR,1725517091
75,75,newman,37.3138,-121.0208,23.35,60,0,1.91,US,1725517132


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
13,priozersk,RU,61.0393,30.1291,78,
31,tongliao,CN,43.6125,122.2653,34,
36,kone,NC,-21.0595,164.8658,44,
38,sao vicente,BR,-23.9631,-46.3919,56,
75,newman,US,37.3138,-121.0208,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : "accommodation.hotel", 
    "limit" : 1,
    "apiKey" : "f9ca043885b84b509b39f1dfa6f6006a"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Print the raw API response for debugging
    print(f"API Response for {row['City']}: {name_address}")

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
API Response for priozersk: {'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Корела', 'country': 'Russia', 'country_code': 'ru', 'region': 'Northwestern Federal District', 'state': 'Leningrad Oblast', 'county': 'Käkisalmi District', 'city': 'Priozersk', 'municipality': 'Приозерское городское поселение', 'postcode': '188760', 'street': 'улица Калинина', 'housenumber': '11', 'lon': 30.1159052, 'lat': 61.0355786, 'formatted': 'Корела, улица Калинина 11, Priozersk, Leningrad Oblast, Russia, 188760', 'address_line1': 'Корела', 'address_line2': 'улица Калинина 11, Priozersk, Leningrad Oblast, Russia, 188760', 'categories': ['accommodation', 'accommodation.hotel'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database License', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Корела', 'osm_id': 833725081, 'tourism': 'hotel', 'osm_type': 

,City,Country,Lat,Lng,Humidity,Hotel Name
13,priozersk,RU,61.0393,30.1291,78,Корела
31,tongliao,CN,43.6125,122.2653,34,No hotel found
36,kone,NC,-21.0595,164.8658,44,Pacifik Appartels
38,sao vicente,BR,-23.9631,-46.3919,56,Pousada Vitória
75,newman,US,37.3138,-121.0208,60,No hotel found
82,ada,US,43.4166,-116.1262,47,No hotel found
93,vila velha,BR,-20.3297,-40.2925,93,Hotel Prainha
101,haiku-pauwela,US,20.9219,-156.3051,73,Inn At Mama's Fish House
109,chengde,CN,40.9725,117.9361,43,紫御国际假日酒店
121,richards bay,ZA,-28.7830,32.0377,76,SUN1 Richards Bay


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.scatter(
    x="Lng", 
    y="Lat", 
    size="Humidity", 
    color="Humidity", 
    hover_cols=["City", "Country", "Hotel Name"], 
    title="Cities and Nearby Hotels", 
    xlabel="Longitude", 
    ylabel="Latitude"
)

# Display the map
map_plot

:Scatter   [Lng]   (Lat,Humidity,City,Country,Hotel Name)